In [127]:
# download dataset
!gdown 1qiUDDoYyRLBiKOoYWdFl_5WByHE8Cugu

Downloading...
From: https://drive.google.com/uc?id=1qiUDDoYyRLBiKOoYWdFl_5WByHE8Cugu
To: c:\Users\ADMIN\OneDrive - VNU-HCMUS\Old Laptop\AIO2024\Main Class\AIO-Exercise\Module_5\Week_3\Auto_MPG_data.csv

  0%|          | 0.00/15.4k [00:00<?, ?B/s]
100%|██████████| 15.4k/15.4k [00:00<?, ?B/s]


In [128]:
# import libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [129]:
# initialize random seed
random_state = 59
np.random.seed(random_state)
torch.manual_seed(random_state)
if torch.cuda.is_available():
    torch.cuda.manual_seed(random_state)

In [130]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [131]:
dataset_path = './Auto_MPG_data.csv'
dataset = pd.read_csv(dataset_path)

In [132]:
X = dataset.drop(columns='MPG').to_numpy()
y = dataset['MPG'].to_numpy()

In [133]:
val_size = 0.2
test_size = 0.125
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size = val_size,
    random_state = random_state,
    shuffle = is_shuffle
)

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train,
    test_size = test_size,
    random_state = random_state,
    shuffle = is_shuffle
)

In [134]:
normalizer = StandardScaler()
X_train = normalizer.fit_transform(X_train)
X_val = normalizer.transform(X_val)
X_test = normalizer.transform(X_test)

X_train = torch.tensor(X_train, dtype = torch.float32)
X_val = torch.tensor(X_val, dtype = torch.float32)
X_test = torch.tensor(X_test, dtype = torch.float32)
y_train = torch.tensor(y_train, dtype = torch.float32)
y_val = torch.tensor(y_val, dtype = torch.float32) 
y_test = torch.tensor(y_test, dtype = torch.float32)

In [135]:
# build pytorch dataset
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [136]:
# build dataloader
batch_size = 32
train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, 
                          batch_size=batch_size, 
                          shuffle= True)
val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=False)

### Train and Evaluation

In [137]:
def r_squared(y_true, y_pred):
    y_true = torch.Tensor(y_true).to(device)
    y_pred = torch.Tensor(y_pred).to(device)
    mean_true = torch.mean(y_true)
    ss_res = torch.sum((y_true - y_pred)**2)
    ss_tot = torch.sum((y_true - mean_true)**2)
    r2 = 1 - ss_res/ss_tot
    return r2.item()

In [138]:
def train_and_evaluate(model, epochs, optimizer, criterion):
    train_losses = []
    val_losses = []
    train_r2 = []
    val_r2 = []
    for epoch in range(epochs):
        train_loss = 0.0
        train_target = []
        train_predict = []
        val_target = []
        val_predict = []
        model.train() # start train
        for X_samples, y_samples in train_loader:
            X_samples = X_samples.to(device)
            y_samples = y_samples.to(device)
            optimizer.zero_grad() # reset grad
            outputs = model(X_samples)
            train_predict += outputs.tolist()
            train_target += y_samples.tolist()
            loss = criterion(outputs, y_samples)
            loss.backward()
            optimizer.step() # update para
            train_loss += loss.item()
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        train_r2.append(r_squared(train_target, train_predict))
        model.eval() # set model to evaluation
        val_loss = 0.0
        with torch.no_grad():
            for X_samples, y_samples in val_loader:
                X_samples = X_samples.to(device)
                y_samples = y_samples.to(device)
                outputs = model(X_samples)
                val_predict += outputs.tolist()
                val_target += y_samples.tolist()
                loss = criterion(outputs, y_samples)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        val_losses.append(val_loss)
        val_r2.append(r_squared(val_target, val_predict))
        print(f'\nEPOCH {epoch + 1}:\tTraining loss: {train_loss: .3f}\tValidation loss: {val_loss: .3f}')

    model.eval()
    with torch.no_grad():
        y_hat = model(X_test)
        test_set_r2 = r_squared(y_hat, y_test)
        print('valuation on test set:')
        print(f'R2: {test_set_r2}')

### MLP 

In [139]:
class MLP(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims, activation):
        super().__init__()
        self.linear1 = nn.Linear(input_dims, hidden_dims)
        self.linear2 = nn.Linear(hidden_dims, hidden_dims)
        self.output = nn.Linear(hidden_dims, output_dims)
        self.activation = activation
    
    def forward(self, x):
        x = self.linear1(x)
        activations = {
            'relu': F.relu,
            'sigmoid': torch.sigmoid,
            'tanh': torch.tanh
        }
        x = activations[self.activation](x)
        x = self.linear2(x)
        x = activations[self.activation](x)
        out = self.output(x)
        return out.squeeze(1)
    
input_dims = X_train.shape[1]
output_dims = 1
hidden_dims = 64
lr = 1e-2

### MLP with ReLU

In [140]:
model = MLP(input_dims, hidden_dims, output_dims, activation='relu').to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = lr)
train_and_evaluate(model, epochs=100, optimizer=optimizer, criterion=criterion)


EPOCH 1:	Training loss:  282.770	Validation loss:  88.669

EPOCH 2:	Training loss:  137.601	Validation loss:  72.725

EPOCH 3:	Training loss:  73.024	Validation loss:  23.437

EPOCH 4:	Training loss:  33.473	Validation loss:  9.792

EPOCH 5:	Training loss:  15.966	Validation loss:  21.951

EPOCH 6:	Training loss:  40.587	Validation loss:  26.677

EPOCH 7:	Training loss:  21.031	Validation loss:  5.269

EPOCH 8:	Training loss:  40.389	Validation loss:  96.423

EPOCH 9:	Training loss:  26.096	Validation loss:  43.621

EPOCH 10:	Training loss:  29.136	Validation loss:  45.975

EPOCH 11:	Training loss:  32.704	Validation loss:  26.076

EPOCH 12:	Training loss:  10.828	Validation loss:  5.663

EPOCH 13:	Training loss:  15.924	Validation loss:  20.072

EPOCH 14:	Training loss:  13.710	Validation loss:  16.768

EPOCH 15:	Training loss:  17.010	Validation loss:  9.460

EPOCH 16:	Training loss:  11.250	Validation loss:  22.420

EPOCH 17:	Training loss:  14.216	Validation loss:  13.509

EPOCH 1

### Linear Regression

In [141]:
class LinearRegression(nn.Module):
    def __init__(self, input_dims, output_dims):
        super().__init__()
        self.output = nn.Linear(input_dims, output_dims)
    
    def forward(self, x):
        return self.output(x).squeeze(1)

model_lr = LinearRegression(input_dims, output_dims).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model_lr.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
train_and_evaluate(model=model_lr, epochs=100, optimizer=optimizer, criterion=criterion)


EPOCH 1:	Training loss:  394.539	Validation loss:  165.868

EPOCH 2:	Training loss:  48.157	Validation loss:  42.470

EPOCH 3:	Training loss:  58.916	Validation loss:  37.810

EPOCH 4:	Training loss:  27.355	Validation loss:  7.610

EPOCH 5:	Training loss:  15.265	Validation loss:  16.799

EPOCH 6:	Training loss:  16.935	Validation loss:  10.642

EPOCH 7:	Training loss:  12.903	Validation loss:  8.280

EPOCH 8:	Training loss:  12.169	Validation loss:  8.075

EPOCH 9:	Training loss:  12.138	Validation loss:  8.364

EPOCH 10:	Training loss:  12.528	Validation loss:  8.572

EPOCH 11:	Training loss:  12.215	Validation loss:  8.489

EPOCH 12:	Training loss:  11.814	Validation loss:  8.409

EPOCH 13:	Training loss:  11.754	Validation loss:  8.341

EPOCH 14:	Training loss:  12.359	Validation loss:  7.954

EPOCH 15:	Training loss:  11.837	Validation loss:  8.401

EPOCH 16:	Training loss:  12.241	Validation loss:  7.981

EPOCH 17:	Training loss:  12.327	Validation loss:  8.685

EPOCH 18:	Train

### MLP with Sigmoid

In [142]:
mlp_sigmoid = MLP(input_dims=input_dims, hidden_dims=hidden_dims, output_dims=output_dims, activation='sigmoid').to(device)
optimizer = torch.optim.SGD(mlp_sigmoid.parameters(), lr = 0.01)
criterion = nn.MSELoss()
train_and_evaluate(mlp_sigmoid, epochs=100, optimizer=optimizer, criterion=criterion)


EPOCH 1:	Training loss:  154.702	Validation loss:  57.432

EPOCH 2:	Training loss:  45.598	Validation loss:  39.028

EPOCH 3:	Training loss:  35.270	Validation loss:  25.857

EPOCH 4:	Training loss:  26.740	Validation loss:  18.978

EPOCH 5:	Training loss:  20.906	Validation loss:  15.328

EPOCH 6:	Training loss:  18.542	Validation loss:  9.103

EPOCH 7:	Training loss:  16.811	Validation loss:  8.437

EPOCH 8:	Training loss:  16.611	Validation loss:  8.375

EPOCH 9:	Training loss:  14.083	Validation loss:  7.835

EPOCH 10:	Training loss:  13.490	Validation loss:  8.019

EPOCH 11:	Training loss:  13.739	Validation loss:  7.358

EPOCH 12:	Training loss:  13.011	Validation loss:  9.474

EPOCH 13:	Training loss:  12.695	Validation loss:  7.588

EPOCH 14:	Training loss:  12.196	Validation loss:  7.079

EPOCH 15:	Training loss:  12.015	Validation loss:  7.032

EPOCH 16:	Training loss:  11.812	Validation loss:  6.951

EPOCH 17:	Training loss:  11.515	Validation loss:  6.790

EPOCH 18:	Traini

### MLP with Tanh

In [143]:
mlp_tanh = MLP(input_dims=input_dims, hidden_dims=hidden_dims, output_dims=output_dims, activation='tanh').to(device)
optimizer = torch.optim.SGD(mlp_tanh.parameters(), lr = 0.01)
criterion = nn.MSELoss()
train_and_evaluate(mlp_tanh, epochs=100, optimizer=optimizer, criterion=criterion)


EPOCH 1:	Training loss:  270.298	Validation loss:  23.794

EPOCH 2:	Training loss:  23.296	Validation loss:  10.683

EPOCH 3:	Training loss:  17.826	Validation loss:  11.399

EPOCH 4:	Training loss:  14.688	Validation loss:  9.336

EPOCH 5:	Training loss:  13.677	Validation loss:  7.972

EPOCH 6:	Training loss:  11.327	Validation loss:  7.513

EPOCH 7:	Training loss:  10.812	Validation loss:  7.302

EPOCH 8:	Training loss:  10.686	Validation loss:  6.366

EPOCH 9:	Training loss:  10.625	Validation loss:  5.676

EPOCH 10:	Training loss:  8.697	Validation loss:  6.100

EPOCH 11:	Training loss:  8.753	Validation loss:  6.475

EPOCH 12:	Training loss:  8.831	Validation loss:  8.572

EPOCH 13:	Training loss:  8.494	Validation loss:  5.058

EPOCH 14:	Training loss:  8.621	Validation loss:  5.805

EPOCH 15:	Training loss:  7.231	Validation loss:  6.339

EPOCH 16:	Training loss:  7.649	Validation loss:  4.847

EPOCH 17:	Training loss:  8.105	Validation loss:  7.179

EPOCH 18:	Training loss:  